## 8.3 Das Newton-Verfahren

**Implementierung 8.2: Newton-Verfahren**

Das Newton-Verfahren können wir folgendermaßen implementieren. Um dabei das Konvergenzverhalten besser studieren zu können, geben wir hier in jedem Schritt Informationen über die aktuelle Approximation der Lösung aus.

In [ ]:
def newton_skalar(f, f_abl, x, n=10, tol=1e-10):
    print('it x          f(x)')
    print(f'0 {x: .8f} {f(x): .4e}')
    for i in range(n):
        x -=  f(x) / f_abl(x)
        print(f'{i + 1} {x: .8f} {f(x): .4e}')
        if abs(f(x)) < tol:
            print(f'Die Newton Methode ist nach {i + 1} Iterationen konvergiert')
            return x
    print(f'Die Newton Methode ist nach {n} Iterationen nicht konvergiert')
    print(f'x = {x}, abs(f(x)) = {abs(f(x))} > {tol}')
    return x

#### Beispiel 8.13

Wir testen den Algorithmus anhand der Funktion
$$f(x) = x(1 + \exp(x)) + 10\sin(3 + \log(x^2 + 1)),$$
dessen Nullstelle wir suchen. Dafür müssen wir wieder die Funktion $f$ und dessen Ableitung definieren.

In [ ]:
from math import exp, log, cos, sin

def f(x):
    return x * (1 + exp(x)) + 10 * sin(3 + log(x**2 + 1))

def fabl(x):
    return (x**2 + 20 * x * cos(log(x**2 + 1) + 3) + 1)/(x**2 + 1) + exp(x) * (x + 1)

Mit dem Startwert $x_0 = 0$ ergibt dies dann

In [ ]:
newton_skalar(f, fabl, 0, n=10, tol=1e-12)

Wir haben die Nullstelle also nach nur 6 Iterationsschritten identifiziert, also weniger als ein Fünftel der Schritte welche die Intervallschachtelung benötigt hat.

Wenn eine Funktion mehrere Nullstellen hat, konvergiert das Newton-Verfahren bei einem anderen Startwert gegebenenfalls gegen eine andere Nullstelle:

In [ ]:
newton_skalar(f, fabl, -10, n=10, tol=1e-12)

Da (vor allem bei vektorwertigen Funktionen), das Invertieren der Ableitung (der Jacobi-Matrix im Höher-Dimensionalen) rechenaufwendig ist, kann es eine gute Idee sein die Abstiegsrichtung einmal fest zu wählen. Dazu passen wir unsere obige Implementierung an.

**Implementierung 8.3: Vereinfachtes Newton-Verfahren**

In [ ]:
def newton_vereinfacht_skalar(f, f_abl, x, c, n=10, tol=1e-10):
    f_abl_x_inv =  1 / f_abl(c)
    
    print('it x           f(x)')
    print(f'0 {x: .8f} {f(x): .4e}')
    for i in range(n):
        fx_alt = f(x)
        x -=  f_abl_x_inv * f(x) 
        print(f'{i + 1:02d} {x: .8f} {f(x): .4e}')
        if abs(f(x)) < tol:
            print(f'Die vereinfachte Newton-Methode ist nach {i + 1} Iterationen konvergiert')
            return x
    print(f'Die Newton-Methode ist nach {n} Iterationen nicht konvergiert')
    print(f'x = {x}, abs(f(x)) = {abs(f(x))} > {tol}')
    return x

#### Beispiel 8.22 (Vereinfachtes Newton-Verfahren)

Angewandt auf das vorherige Beispiel mit $c=x_0$ und der größeren Toleranz $\epsilon=10^{-7}$ ergibt dies dann

In [ ]:
newton_vereinfacht_skalar(f, fabl, -10, -10, n=50, tol=1e-7)

Das Verfahren konvergiert also nur recht langsam. Bei einer geschickter Wahl der Stelle der Ableitung, kann die Methode auch etwas schneller konvergieren. Wenn Sie die Stelle aber falsch setzen, konvergiert das Verfahren gegebenenfalls auch gar nicht. Probieren Sie es aus!

In [ ]:
newton_vereinfacht_skalar(f, fabl, -10, -9.344, n=50, tol=1e-7)

Statt die Abstiegsrichtung fest zu wählen, können wir auch die Ableitung numerisch approximieren.

**Implementierung 8.4: Approximiertes Newton-Verfahren**:

In [ ]:
def newton_approx_skalar(f, x, eps, n=10, tol=1e-10):
    print('it x          f(x)')
    print(f'0 {x: .8f} {f(x): .4e}')
    for i in range(n):
        y = f(x)
        z = f(x + eps)
        x -=  eps * y / (z - y)
        print(f'{i + 1} {x: .8f} {f(x): .4e}')
        if abs(f(x)) < tol:
            print(f'Die approximierte Newton-Methode ist nach {i + 1} Iterationen konvergiert')
            return x
    print(f'Die approximierte Newton-Methode ist nach {n} Iterationen nicht konvergiert')
    print(f'x = {x}, abs(f(x)) = {abs(f(x))} > {tol}')
    return x

#### Beispiel 8.23 (Approximiertes Newton-Verfahren)

Hier hängt die Konvergenz des Verfahrens an der korrekten Wahl von $\epsilon$ in der finite Differenzen Approximation ab: 

In [ ]:
for eps in [1e-1, 1e-2, 1e-4, 1e-8, 1e-12, 2e-14]:
    print('\n------------------------')
    print(f'eps = {eps}\n')
    newton_approx_skalar(f, -10, eps, n=10, tol=1e-10)

Abschließend betrachten wir noch zwei weitere, aber eher wenig verbreitete Verfahren zur Bestimmung von Nullstellen.

**Algorithmus 7.8: Sekantenverfahren**

In [ ]:
def sekantenverfahren(f, x0, x1, n=10, tol=1e-12):
    f0 = f(x0)
    for i in range(n):
        f1 = f(x1)
        if abs(f1) < tol:
            print(f'Das Sekantenverfahren ist nach {i + 1} Iterationen konvergiert')
            return x1
        
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        x0, f0 = x1, f1
        x1 = x2
    print(f'Das Sekantenverfahren ist nach {n} Iterationen nicht konvergiert')
    print(f'x = {x1}, abs(f(x)) = {abs(f1)} > {tol}')
    return x1

Angewandt auf unser bekanntes Beispiel ergibt dies mit $x_0=-10$ und $x_1=-9$

In [ ]:
sekantenverfahren(f, -10, -9, n=10, tol=1e-12)

**Algorithmus 7.9: Regula falsi**

In [ ]:
def regula_falsi(f, x0, x1, n=10, tol=1e-12):
    f0 = f(x0)
    f1 = f(x1)
    for i in range(n):
        
        if abs(f1) < tol:
            print(f'Die Regula falsi-Methode ist nach {i + 1} Iterationen konvergiert')
            return x1
        
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        f2 = f(x2)
        if f2 * f1 < 0:
            x0, f0 = x1, f1
            x1, f1 = x2, f2
        else:
            x1, f1 = x2, f2
    print(f'Die Regula falsi-Methode ist nach {n} Iterationen nicht konvergiert')
    print(f'x = {x1}, abs(f(x)) = {abs(f1)} > {tol}')
    return x1

Mit denselben Parametern wie für die Sekantenmethode erhalten wir dann

In [ ]:
regula_falsi(f, -10, -9, n=50, tol=1e-12)

Wir sehen also, dass die schnelle Konvergenzgeschwindigkeit der Sekantenmethode in der Tat verloren gegangen ist.